In [3]:
# fetch input text
import urllib.request
url = 'https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt'
file_path = 'the-verdict.txt'
urllib.request.urlretrieve(url, file_path)

with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:100])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [9]:
# split text into tokens
import re
preprocessed = [item for item in re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [21]:
# convert tokens into token IDs

# create vocabulary
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break
    

1132
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [29]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {id:token for token,id in vocab.items()}
    
    def encode(self, text):
        preprocessed = [token.strip() for token in re.split(r'([,.:;?_!"()\']|--|\s)', text) if token.strip()]
        return [self.str_to_int[s] for s in preprocessed]
    
    def decode(self, ids):
        # "Hello , world !"  →  "Hello, world!"
        return re.sub(r'\s+([,.?!"()\'])', r'\1', " ".join([self.int_to_str[id] for id in ids]))

tokenizer = SimpleTokenizerV1(vocab)
text = 'I turned to Mrs. Gisburn, who had lingered to give a lump of sugar to her spaniel in the dining-room.'
print(tokenizer.decode(tokenizer.encode(text)) == text)


True
1132
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)
True
<|unk|> Mrs.
<|unk|> Mrs.


In [33]:

# add special token for vocabulary
all_tokens = all_words + ['<|endoftext|>', '<|unk|>']
vocab_size = len(all_tokens)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(all_tokens)}
for i, item in enumerate(vocab.items()):
    if i > len(vocab) - 5:
        print(item)

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {id:token for token,id in vocab.items()}
    
    def encode(self, text):
        preprocessed = [token.strip() for token in re.split(r'([,.:;?_!"()\']|--|\s)', text) if token.strip()]
        preprocessed = [token if token in self.str_to_int else '<|unk|>' for token in preprocessed]
        return [self.str_to_int[s] for s in preprocessed]
    
    def decode(self, ids):
        # "Hello , world !"  →  "Hello, world!"
        return re.sub(r'\s+([,.?!"()\'])', r'\1', " ".join([self.int_to_str[id] for id in ids]))

tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

1132
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
